In [1]:
import os
from dotenv import load_dotenv
load_dotenv() #get environment variables from .env file

TL_API_KEY = os.getenv("API_KEY")
assert TL_API_KEY


### Instiantiate TwelveLabs Client with API KEY

In [2]:
from twelvelabs import TwelveLabs

client = TwelveLabs(api_key=TL_API_KEY)

### Create an Index  (ONLY RUN ONCE)

In [ ]:
from twelvelabs import APIStatusError

try:
    print("Creating Index... \n")
    index = client.index.create(
        name = "123",
        engines = [{"name" : "marengo2.6", "options" : ["visual", "conversation", "text_in_video"]},
                   {"name" : "pegasus1.1", "options" : ["visual", "conversation"]}], # We dont use Pegasus
        
         addons=["thumbnail"]
    )
    
    print(f"Created index: id={index.id} name={index.name} engines={index.engines}")
    
except APIStatusError as e:
    print("API Status Error, 4xx or 5xx : ",e)
    
except Exception as e:
    print(e)


Helper functions

In [4]:
import math

def convert_seconds(seconds:str):
    """Converts seconds to hours, minutes, and seconds in format HH:MM:SS or MM:SS"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds_remaining = math.floor(seconds % 60)
    
    if hours > 0:
        return f"{hours}:{minutes:02}:{seconds_remaining:02}"
    else:
        return f"{minutes}:{seconds_remaining:02}"
    
    
def retrieve_video_name(index_id: str, video_id :str):
    """Returns file name of video taking index_id and video_id as arguments"""
    video_content = client.index.video.retrieve(index_id=index_id, id=video_id)
    video_name = video_content.metadata.filename
    
    return video_name


List all my index available with their id, name and engines used

In [8]:
"""List all my indexes"""
indexes = client.index.list().root
for index in indexes:
    index_id = index.id
    index_name = index.name
    index_engines = index.engines.root
    
    print("Index ID:", index_id)
    print("Index Name:", index_name)
    print("Index Engines:", [f"Engine: {x.name} has options: {x.options}" for x in index_engines])
    print("---")

Index ID: 6686e98c139f029878ae0e08
Index Name: 123
Index Engines: ["Engine: pegasus1.1 has options: ['visual', 'conversation']", "Engine: marengo2.6 has options: ['visual', 'conversation', 'text_in_video']"]
---
Index ID: 66844735eb3979a66bdc9f94
Index Name: IndexfromMacBook
Index Engines: ["Engine: pegasus1.1 has options: ['visual']", "Engine: marengo2.6 has options: ['visual']"]
---
Index ID: 66841d79eb3979a66bdc9f30
Index Name: Index1
Index Engines: ["Engine: pegasus1.1 has options: ['visual', 'conversation']", "Engine: marengo2.6 has options: ['visual', 'conversation', 'text_in_video', 'logo']"]
---
Index ID: 66841c757e3a05b242e8b122
Index Name: My Index (Default)
Index Engines: ["Engine: pegasus1 has options: ['visual', 'conversation']", "Engine: marengo2.6 has options: ['visual', 'conversation', 'text_in_video', 'logo']"]
---


In [6]:
INDEX_ID = "6686e98c139f029878ae0e08" #add a valid index id

To Upload a video:

In [ ]:
"""Use this to upload video to an index"""
from glob import glob
from twelvelabs.models.task import Task
from twelvelabs import BadRequestError

video_files = glob("./vids/*.mp4") # path to videos

for vid in video_files:
    print(f"Uploading {vid}")
    try:
        task = client.task.create(index_id= INDEX_ID, file=vid, language="en")
        print(f"Task id={task.id}\n")
    except BadRequestError as e:
        print(f"Error with task: {e}")
    
# Monitor the video indexing process 
def on_task_update(task: Task):
    """Utility function to print the status of a video indexing task"""
    print(f"  Status={task.status}")
    
try:        
    task.wait_for_done(callback=on_task_update)
    if task.status != "ready":
        raise RuntimeError(f"Indexing failed with status {task.status}")
    print(f"Uploaded - {task.metadata['filename']} - . The unique identifer of your video is {task.video_id}.")
except Exception as e:
    print(f"Error : {e}")
    

List all videos in your index

In [7]:

videos = client.index.video.list(INDEX_ID)

for vid in videos:
    video_id = vid.id
    video_name = vid.metadata.filename
    video_length = vid.metadata.duration
    
    print("Video ID:", video_id)
    print("Video Name:", video_name)
    print("Video length:",convert_seconds(video_length))
    print()

Video ID: 6686eaead22b3a3c97bf3e28
Video Name: Reporters Asking NBA Players Stupid Questions.mp4
Video length: 12:29

Video ID: 6686e9e7d22b3a3c97bf3e27
Video Name: LBJ.mp4
Video length: 4:18

Video ID: 6686e9e5d22b3a3c97bf3e26
Video Name: NBA Most Iconic Commentary Moments Of All Time.mp4
Video length: 8:07

Video ID: 6686e9e2d22b3a3c97bf3e25
Video Name: plays.mp4
Video length: 10:00



In [262]:
#Search through video
text_query = "player shooting a three"

#Note query_text parameter doesn't perform well returns videos as having high scores
#Note adjust_confidence_level does not have any effects
result = client.search.query(index_id=index_id,
                             options=["conversation","visual"], #modify parameters as needed see docs
                             query=text_query,
                             group_by="clip",
                             threshold="medium",
                             page_limit=10 #limit to top 10 results
                             )


In [263]:

index_id = result.pool.index_id

print(f"Here are some results of the text query '{text_query}'")
print("-------------------------------------------------------\n")

for clip in result.data:
      print(
      f"""video name = {retrieve_video_name(index_id=index_id,video_id=clip.video_id)}  
            start_time = {convert_seconds(clip.start)}
            end_time = {convert_seconds(clip.end)} 
            score = {round(clip.score)} 
            confidence = {clip.confidence}\n"""
        )

# #Loop through page and display more results if available      
# while True:
#     try:
#         next_page_data = next(result)
#         print(f"Next page's data \n")
#         for clip in next_page_data:
#             print(
#             f"""video name = {retrieve_video_name(index_id=index_id,video_id=clip.video_id)}
#             start_time = {convert_seconds(clip.start)}
#             end_time = {convert_seconds(clip.end)}   
#             score = {round(clip.score)} 
#             confidence = {clip.confidence}\n"""
#             )
#     except StopIteration:
#         print("\nThere is no more pages in search result")
#         break


Here are some results of the text query 'shooting a three'
-------------------------------------------------------

video name = NBA Most Iconic Commentary Moments Of All Time.mp4  
            start_time = 6:25
            end_time = 7:15 
            score = 85 
            confidence = high

video name = Reporters Asking NBA Players Stupid Questions.mp4  
            start_time = 3:28
            end_time = 3:34 
            score = 84 
            confidence = high

video name = plays.mp4  
            start_time = 1:42
            end_time = 2:43 
            score = 84 
            confidence = high

video name = plays.mp4  
            start_time = 0:20
            end_time = 0:54 
            score = 83 
            confidence = high

video name = NBA Most Iconic Commentary Moments Of All Time.mp4  
            start_time = 4:53
            end_time = 5:10 
            score = 83 
            confidence = high

video name = plays.mp4  
            start_time = 2:46
            